In [ ]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR

from sklearn.metrics import r2_score

from datetime import datetime, timedelta
from matplotlib import pyplot as plt
from matplotlib import dates as mpl_dates

import requests
import json

from statistics import mean
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

In [ ]:
commodity_dict = {
    "alasande_gram" : "newDataset/alasande_gram.csv",
    "arecanut" : "newDataset/arecanut.csv",
    "arhar" : "newDataset/arhar.csv",
    "banana" : "newDataset/banana.csv",
    "banana_green" : "newDataset/banana_green.csv",
    "beans" : "newDataset/beans.csv",
    "black_pepper" : "newDataset/black_pepper.csv",
    "coconut" : "newDataset/coconut.csv",
    "onion" : "newDataset/onion.csv",
    "potato" : "newDataset/potato.csv"
}

crop_selection = 0
variety_selection = 0
feature_selection = 0
market_selection = 0

crop_name_list = ['Alasande Gram', 'Arecanut', 'Arhar', 'Banana', 'Banana Green', 'Beans', 'Black Pepper', ' Coconut', 'Onion', 'Potato']

marketName = ['Bangalore', 'Channapatana', 'Doddaballa Pur', 'Hoskote', 'Kanakapura', 'Ramanagara']


market_list = [['Bangalore', 'Hoskote', 'Kanakapura', 'Ramanagara'],
               ['Bangalore', 'Doddaballa pur'], 
               ['Bangalore', 'Doddaballa pur', 'Hoskote', 'Kanakapura'],
               ['Bangalore', 'Channapatana', 'Hoskote', 'Kanakapura', 'Ramanagara'],
               ['Bangalore', 'Doddaballa pur', 'Hoskote', 'Kanakapura', 'Ramanagara'],
               ['Bangalore', 'Channapatana', 'Doddaballa pur', 'Hoskote', 'Kanakapura', 'Ramanagara'],
               ['Bangalore'],
               ['Bangalore', 'Channapatana', 'Doddaballa pur', 'Kanakapura', 'Ramanagara'],
               ['Bangalore', 'Channapatana', 'Doddaballa pur', 'Hoskote', 'Kanakapura', 'Ramanagara'],
               ['Bangalore', 'Channapatana', 'Doddaballa pur', 'Hoskote', 'Kanakapura', 'Ramanagara']
              ]


variety_list = [['Alasande Gram'], 
                ['Other', 'Pudi'],
                ['Arhar Dal(Tur)'], 
                ['Elakki Bale', 'Medium', 'Nauti Bale', 'Nendra Bale', 'Other', 'Pachha Bale', 'Rasa Bale'],
                ['Banana - Green'],
                ['Beans (Whole)'],
                ['Other'],
                ['Coconut', 'Grade-I', 'Imported', 'Other'],
                ['Bangalore-Samall', 'Local', 'Onion', 'Other', 'Puna'],
                ['Chips', 'Kuber', 'Local', 'Potato']
               ]

commodity_list = []

avg_accuracy_RF = []
avg_accuracy_DT = []
avg_accuracy_SVM = []

single_market_prices = []
marketPrices = []

In [ ]:
class Commodity:
    
    def __init__(self, crop_name):
        self.name = crop_name
        dataset = pd.read_csv(crop_name)
        self.X = dataset.iloc[:, 0:5].values
        self.y = dataset.iloc[:, 5].values
        
        self.y = self.y.reshape(len(self.y),1)
        
        #Encoding
        self.ct = ColumnTransformer(transformers=[('encoder', OrdinalEncoder(), [0,1])], remainder='passthrough')
        self.X = np.array(self.ct.fit_transform(self.X))
        
        #spliting the dataset into train and test set
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size = 0.1, random_state = 0)
        
        #Random Forest 
        self.y_train_RF = self.y_train.ravel() 
        self.regressor_RF = RandomForestRegressor(n_estimators = 100, random_state = 0)
        self.regressor_RF.fit(self.X_train, self.y_train_RF)
        self.y_pred_RF = self.regressor_RF.predict(self.X_test)
        self.accuracy_RF = r2_score(self.y_test, self.y_pred_RF)
        avg_accuracy_RF.append(self.accuracy_RF)
        
        #Decision Tree
        self.regressor_DT = DecisionTreeRegressor(random_state = 0)
        self.regressor_DT.fit(self.X_train, self.y_train_RF)
        self.y_pred_DT = self.regressor_DT.predict(self.X_test)
        self.accuracy_DT = r2_score(self.y_test,self. y_pred_DT)
        avg_accuracy_DT.append(self.accuracy_DT)
        
        #SVM
        self.sc_X = StandardScaler()
        self.sc_y = StandardScaler()
        self.X_train_SVM = self.sc_X.fit_transform(self.X_train)
        self.y_train_SVM = self.sc_y.fit_transform(self.y_train)

        
        self.regressor_SVM = SVR(kernel = 'rbf')
        self.regressor_SVM.fit(self.X_train_SVM, self.y_train_SVM)

        self.n_y_pred = self.regressor_SVM.predict(self.sc_X.transform(self.X_test))
        self.n_y_pred = self.n_y_pred.reshape(len(self.n_y_pred),1)

        self.y_pred = self.sc_y.inverse_transform(self.n_y_pred)
        self.accuracy_SVM = r2_score(self.y_test, self.y_pred)
        avg_accuracy_SVM.append(self.accuracy_SVM)
        
        
        #marketName = dataset['Market Name'].unique().tolist()
        #dataset['Market Name'].value_counts()
        #print(marketName)
        
        
    def modelPerformance(self):
        data = self.X_test[10:20, :]
        actual_result = self.y_test[10:20, :]
        pred_result_RF = []
        pred_result_DT = []
        pred_result_SVM = []
        pred_result_SVM_translate = []

        check_day_month_year = []

        for c, v, d, m, y in data:
            check_day_month_year.append(str(y) + '-' + str(m) + '-' + str(d))

        check_day_month_year.sort()

        dates = [datetime.strptime(x, '%Y-%m-%d') for x in check_day_month_year]   # creates a datetime object
        date_label = [x.strftime("%b %Y %d") for x in dates]                       # convert datetime object to string format

        for i in range(len(data)):
            pred_result_RF.append(self.regressor_RF.predict([data[i]]).tolist())

        for i in range(len(data)):
            pred_result_DT.append(self.regressor_DT.predict([data[i]]).tolist())

        for i in range(len(data)):
            pred_result_SVM.append(self.regressor_SVM.predict(self.sc_X.transform([data[i]])).tolist())
        pred_result_SVM = self.sc_y.inverse_transform(pred_result_SVM)
        
        
        print('Model Accuracy: \n1. Random Forest ==> ', round(self.accuracy_RF*100, 2), '%\n2. Decision Tree ==> ', round(self.accuracy_DT*100, 2), '%\n3. Secure Vector Machine ==> ', round(self.accuracy_SVM*100, 2), '%')
        

        plt.style.use('seaborn')

        fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, ncols=1)

        ax1.plot_date(check_day_month_year, actual_result, linestyle='solid', color='blue', label='Actual value', alpha=0.75)
        ax1.plot_date(check_day_month_year, pred_result_RF, linestyle='solid', color='red', label='Predicted Value(Random Forest)', alpha=0.75)
        ax1.set_title('Model Preformance')
        ax1.legend()


        ax2.plot_date(check_day_month_year, actual_result, linestyle='solid', color='blue', label='Actual value', alpha=0.75)
        ax2.plot_date(check_day_month_year, pred_result_DT, linestyle='solid', color='red', label='Predicted Value(Decision Tree)', alpha=0.75)
        ax2.set_xlabel('Date')
        ax2.set_ylabel('Model Price')
        ax2.legend()

        ax3.plot_date(check_day_month_year, actual_result, linestyle='solid', color='blue', label='Actual value', alpha=0.75)
        ax3.plot_date(check_day_month_year, pred_result_SVM, linestyle='solid', color='red', label='Predicted Value(SVM)', alpha=0.75)
        plt.gcf().autofmt_xdate()                                       #rotate date to right align at x axis.
        plt.xticks(ticks=check_day_month_year, labels=date_label)

        ax3.set_xlabel('Date')
        ax3.legend()
        plt.tight_layout()
        #plt.savefig('Performance.png')
        plt.show()
        
    def getPrediction(self, mar, var, d, m, y):
        return self.regressor_RF.predict([[mar, var, d, m, y]])
    

    def twelve_month_forecast(self, market_name, variety, day=15):
        
        current_month = datetime.now().month
        current_year = datetime.now().year

        month_with_year = []
        day_month_year = []

        for i in range(1, 13):
            if current_month + i <= 12:
                month_with_year.append((current_month + i, current_year))
                day_month_year.append(str(current_year) + '-' + str(current_month + i) + '-' + str(day))
            else:
                month_with_year.append((current_month + i -12, current_year + 1))
                day_month_year.append(str(current_year + 1) + '-' + str(current_month + i - 12) + '-' + str(day))

        dates = [datetime.strptime(x, '%Y-%m-%d') for x in day_month_year]
        dates.sort()
        date_label = [x.strftime("%b %Y") for x in dates]

        prediction = []

        for m, y in month_with_year:
            prediction.append(self.regressor_RF.predict([[market_name, variety, day, m, y]]).tolist())

        for i in range(len(prediction)):
            print(day , ' /', month_with_year[i][0] , '/' , month_with_year[i][1] , ' --- >' , prediction[i][0])
        
        
        # For Plotting the graph
        
        crop_name_for_graph = crop_name_list[crop_selection]  #crop name
        plt.style.use('seaborn')                              # uses the specifies graph
        plt.plot_date(day_month_year, prediction, label=crop_name_for_graph , linestyle='solid', alpha=0.5)
        plt.gcf().autofmt_xdate()                             # it will get the current figure and rotate the date to look nice
        plt.xticks(ticks=day_month_year, labels=date_label)   # for change labels in x-axis
        plt.title('Twelve Month Forecast')
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.tight_layout()                                    # adds [padding to our plots]
        plt.legend()
        plt.savefig('pred.png')
        plt.show()
        
  
          
    def current_day(self):
        current_day = datetime.now().day
        current_month = datetime.now().month
        current_year = datetime.now().year

        current_price = self.regressor_RF.predict([[float(market_selection), float(variety_selection), current_day, current_month, current_year]])[0]

        print('The current price of ',crop_name_list[crop_selection], '(',variety_list[crop_selection][variety_selection], ') in ',marketName[market_selection], ' is: ₹', current_price, '/Quintal')
    
    
    def bestMarketOnGivenDate(self, variety, day, month, year):
        bestMarket = []

        for i in range(len(market_list[crop_selection])):
            bestMarket.append(self.regressor_RF.predict([[float(i), variety, day, month, year]]).tolist())

        for i in range(len(bestMarket)):
            print(market_list[crop_selection][i], " --->  ₹", bestMarket[i][0], '/Quintal')
            
            
    def distance(self, source,destination):

        url = "https://maps.googleapis.com/maps/api/distancematrix/json?origins="+source+"&destinations="+destination+"&transit_mode=transit&transit_mode=bus&key=PASTE_YOUR_API_KEY"
        response = requests.get(url)


        data = json.loads(response.text)                
        a=(data['rows'][0]['elements'][0]['distance']['text'].split(' '))[0]
        return float(a)
    
    
    def numberOfDays(self, y, m):
        leap = 0
        if y% 400 == 0:
            leap = 1
        elif y % 100 == 0:
            leap = 0
        elif y% 4 == 0:
            leap = 1
        if m==2:
            return 28 + leap
        list = [1,3,5,7,8,10,12]
        if m in list:
            return 31
        return 30
    
    
    def bestSellingPrice(self, marketPrices, month, year, source):
        print('\n')
        for i in range(len(marketPrices)):
            fare = self.distance(source, market_list[crop_selection][i])
            top = np.argmax(marketPrices[i])
            print('Market Name ---> ', market_list[crop_selection][i])
            print('Distance from ', source, ' to ', market_list[crop_selection][i], ' ---> ', fare, 'Km')
            print('Best Price on ', top+1, '/',month, '/', year ,  " --> ₹", marketPrices[i][top], '\n')
            print('-' * 50, '\n')
            
    def topMarketsPricesOnGivenMonth(self, marketName, variety, month, year, source):
        
        marketPrices.clear()
        
        for m in range(len(marketName)):
            fare = float(self.distance(source, marketName[m]) * 1.8)
            single_market_prices = []
            for d in range(1, self.numberOfDays(year, month) + 1):
                single_market_prices.append(self.regressor_RF.predict([[float(m), variety, d, month, year]]).tolist()[0] - fare)
                
            marketPrices.append(single_market_prices)
            
        self.bestSellingPrice(marketPrices, month, year, source)

In [ ]:
if __name__ == "__main__":
    alasande_gram = Commodity(commodity_dict["alasande_gram"])
    commodity_list.append(alasande_gram)
    
    arecanut = Commodity(commodity_dict["arecanut"])
    commodity_list.append(arecanut)
    
    arhar = Commodity(commodity_dict["arhar"])
    commodity_list.append(arhar)
    
    banana = Commodity(commodity_dict["banana"])
    commodity_list.append(banana)
    
    banana_green = Commodity(commodity_dict["banana_green"])
    commodity_list.append(banana_green)
    
    beans = Commodity(commodity_dict["beans"])
    commodity_list.append(beans)
    
    black_pepper = Commodity(commodity_dict["black_pepper"])
    commodity_list.append(black_pepper)
    
    coconut = Commodity(commodity_dict["coconut"])
    commodity_list.append(coconut)
    
    onion = Commodity(commodity_dict["onion"])
    commodity_list.append(onion)
    
    potato = Commodity(commodity_dict["potato"])
    commodity_list.append(potato)

In [ ]:
def average_accuracy():
    print('1. Average accuracy of Random Forest ==> ', round(mean(avg_accuracy_RF)*100, 2),'%')
    print('2. Average accuracy of Decision Tree ==> ', round(mean(avg_accuracy_DT)*100, 2),'%')
    print('3. Average accuracy of Secure Vector Machine ==> ', round(mean(avg_accuracy_SVM)*100, 2),'%')

In [ ]:
average_accuracy()

In [ ]:
def crop_selections():
    global crop_selection
    print('Select the crop: \n1. Alasande Gram\n2. Arecanut\n3. Arhar\n4. Banana\n5. Banana Green\n6. Beans\n7. Black Pepper\n8. Coconut\n9. Onion\n10. Potato')
    print('\n')
    crop_selection = int(input('Your Selection: ')) - 1
    print('\n')

In [ ]:
def features():
    global feature_selection
    
    while feature_selection != 4:
        
        print('\n\n')
        print('What type of Information is Required:\n1. Current day prediction\n2. Price Prediction of all Market on given Date\n3. Twelve Months Forcast\n4. Top Markets Prices on given Month (Transportations charges included)\n5. Exit')
        print('\n')
        feature_selection = int(input('Your Selection: ')) - 1
        print('\n')

        crop_name = crop_name_list[crop_selection]

        if feature_selection == 0:
            print('Select the Market: ')
            market_selections()

            commodity_list[crop_selection].current_day()

        if feature_selection == 1:

            day, month, year = input('Enter the date on which you need the Predictoin: ').split()

            commodity_list[crop_selection].bestMarketOnGivenDate(float(variety_selection), int(day), int(month), int(year))

        if feature_selection == 2:
            print('Select the Market: ')
            market_selections()

            print('Forcasting for', market_list[crop_selection][market_selection], 'for next 12 months.')
            commodity_list[crop_selection].twelve_month_forecast(float(market_selection), float(variety_selection))

        if feature_selection == 3:
            source = input('Enter your village/town/city name: ')
            month, year = input('Enter the month and year on which you need predictoins: ').split()
            commodity_list[crop_selection].topMarketsPricesOnGivenMonth(market_list[crop_selection], float(variety_selection), int(month), int(year), source)

In [ ]:
def market_selections():
    global market_selection
    for i in range(len(market_list[crop_selection])):
        print(i + 1,'. '  ,market_list[crop_selection][i])
    
    print('\n')
    market_selection = int(input('Your Selection: ')) - 1
    print('\n')

In [ ]:
def variety_selections():
    global variety_selection
    for i in range(len(variety_list[crop_selection])):
        print(i+1,'. ', variety_list[crop_selection][i])
    print('\n')
    variety_selection = int(input('Your Selection: ')) - 1
    print('\n')

In [ ]:
def selection(): 
    crop_selections()
    variety_selections()
    features()            

In [ ]:
def main_function():
    print('-'*5,'-*'*10,'Crop Price Forcating Using Machine Learning','-*'*10,'-'*5,'\n')
    #print('1. Model Performance\n2. Top 5 Gainers\n3. Top 5 Lossers\n4. Predictions\n')
    print('1. Model Performance\n2. Predictions\n')
    choice = int(input('Your Choice: '))
    
    if choice == 1:
        print('\n')
        crop_selections()
        print('\nModel performance for ',crop_name_list[crop_selection], 'dataset is: \n')
        commodity_list[crop_selection].modelPerformance()
        
    if choice == 2:
        selection()

In [ ]:
main_function()